# Unveiling Network Threats: Anomaly Detection in Intrusion Detection Systems

**Author:** Mário Antunes (mario.antunes@ua.pt)

**Date:** 24/11/2023

This notebook explores unsupervised methods to flag malicious packet flows.

The dataset was gathered by the [GCS](https://www.ua.pt/pt/ciberseguranca/sobre-gcs).


In [ ]:
%matplotlib inline
import gzip
import requests
import numpy as np
import polars as pl
import geopy.distance
import seaborn as sns
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

from sklearn.utils import resample
from sklearn.cluster import KMeans
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import LocalOutlierFactor
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from keras import backend as K
from tqdm.keras import TqdmCallback

plt.style.use('ggplot')
plt.rcParams['figure.autolayout'] = True

## Load and Enhance the dataset. 

In [ ]:
#df = pl.read_csv('https://cloud.hrun.duckdns.org/s/ZbwzSwEPk6pEds2/download/user-asn-sorted.csv', null_values=['-'])

f=gzip.open('user-asn-sorted.csv.gz','rb')
file_content=f.read()
df = pl.read_csv(file_content, null_values=['-'])

df = df.with_columns(pl.col('timestamp').str.to_datetime('%Y-%m-%dT%H:%M:%S'))
df = df.drop('malware_type')
df.head()

In [ ]:
signatures = df[['signature']].unique()['signature'].to_list()
dict_anomalies = {}
for k in signatures:
    if k is not None:
        dict_anomalies[k] = 1

df = df.with_columns(pl.col('signature').map_dict(dict_anomalies, default=0).alias('anomaly'))
df = df.drop('signature')
df.head()

In [ ]:
#r = requests.get('https://iptoasn.com/data/ip2asn-v4-u32.tsv.gz', headers={'User-agent': 'Mozilla/5.0'})
#ASNs = pl.read_csv(r.content, separator='\t', has_header=False, new_columns=['start.ip','stop.ip','asn','country.code2','entity'])

f=gzip.open('ip2asn-v4-u32.tsv.gz','rb')
file_content=f.read()
ASNs = pl.read_csv(file_content, separator='\t', has_header=False, new_columns=['start.ip','stop.ip','asn','country.code2','entity'])

ASNs = ASNs.filter(pl.col('asn') != 0)
ASNs = ASNs[['asn','country.code2','entity']].unique()
ASNs.head()

In [ ]:
df = df.join(ASNs, on='asn')
df.head()

In [ ]:
#countries = pl.read_csv('https://cloud.hrun.duckdns.org/s/qDCMSH5HbHKGszE/download/countries_codes_and_coordinates.csv', null_values=['-'])
countries = pl.read_csv('countries_codes_and_coordinates.csv', null_values=['-'])
countries = countries.drop(['country.code3','country.code.num'])
countries.head()

In [ ]:
countries_list = countries.rows()
number_countries = len(countries_list)
countries_distance = np.zeros((number_countries, number_countries))

for i in range(number_countries-1):
    for j in range(i, number_countries):
        coords_1 = (countries_list[i][2], countries_list[i][3])
        coords_2 = (countries_list[j][2], countries_list[j][3])
        dist = geopy.distance.geodesic(coords_1, coords_2).km
        countries_distance[i][j] = dist
        countries_distance[j][i] = dist

largest_distance = np.max(countries_distance)

np.fill_diagonal(countries_distance, largest_distance)

countries_distance

In [ ]:
countries_rank = [0] * number_countries

COUNTRY_IDX = 0
for i in range(number_countries):
    if countries_list[i][1] == 'PT':
        COUNTRY_IDX = i
        break
COUNTRY_IDX

rank = 0
processed_countries = [COUNTRY_IDX]
countries_rank[COUNTRY_IDX] = rank
rank += 1

In [ ]:
while len(processed_countries) < number_countries:
    distances = np.zeros(number_countries)
    for cidx in processed_countries:
        distances += countries_distance[cidx]
    COUNTRY_IDX = np.argmin(distances)

    while COUNTRY_IDX in processed_countries:
        distances[COUNTRY_IDX] = float('inf')
        COUNTRY_IDX = np.argmin(distances)

    processed_countries.append(COUNTRY_IDX)
    countries_rank[COUNTRY_IDX] = rank
    rank += 1

In [ ]:
map_contries = {}

for i in range(number_countries):
    map_contries[countries_list[i][0]] = countries_rank[i]


In [ ]:
map_dst_ports = {0:0, 1:0, 5:0,
7:10, 20:20, 21:20, 22:20, 23:20,
25:40, 37:10, 43:10, 53:10, 80:30,
81:50, 82:20, 83:0, 88: 20, 110:40,
119:40, 123:10, 135:10, 137:10, 139:10,
143:40, 161:10, 389:10, 443:30, 444:30, 445:20,
465:30, 500:10, 514:20, 585:40, 587:40, 843:10,
993:40, 995:40}

In [ ]:
df = df.join(countries, on='country.code2')
df = df.rename({'country.code2': 'country_code'})
df.head()

In [ ]:
df['user'].unique()

In [ ]:
users = ['0028e812', 'cb8f966a', '6b6e3de0', 'f80b9c1b', '769d71e8', 'cd46ec92', '03aec668']

connections = []

for u in users:
    single_user = df.filter(pl.col('user') == u)
    connections.append(single_user.rows(named=True))

In [ ]:
for c in connections:
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.stock_img()

    for i in range(len(c)-1):
        start = c[i]
        stop = c[i+1]
        plt.plot([start['lon'], stop['lon']], [start['lat'], 
        stop['lat']], color='blue', linewidth=1, marker='o', 
        transform=ccrs.Geodetic())
    plt.show()

## Simple Classifier (Logistic Regression)

In [ ]:
df_clean = df[['user', 'dst_port', 'country', 'anomaly']]
df_clean.head()

### Domain Specific Encoding

The communications ports were grouped together into classes (for known ports below 1024) and adjusted accordingly.
Regarding the countries (that are aligned with ASN entity), they are ranked together based on proximity.

In [ ]:
df_unique_user = df.select(pl.col('user').unique(maintain_order=True)).with_row_count(name='user_enc')
df_clean = df_clean.join(df_unique_user, on='user')
df_clean = df_clean.drop('user')
df_clean = df_clean.rename({'user_enc': 'user'})
df_clean = df_clean.with_columns(pl.col('dst_port').map_dict(map_dst_ports, default=100).alias('dst_port'))
df_clean = df_clean.with_columns(pl.col('country').map_dict(map_contries, default=1000).alias('country'))
df_clean.head()

### Full Dataset

In [ ]:
df_malicious = df_clean.filter(pl.col('anomaly') == 1)
df_benign    = df_clean.filter(pl.col('anomaly') == 0)

df_count = pl.DataFrame({'anomaly': ['Benign', 'Malicious'], 'count': [df_benign.shape[0], df_malicious.shape[0]]})
sns.barplot(x='anomaly', y='count', hue='anomaly', data=df_count)
plt.show()

In [ ]:
Y = df_clean['anomaly']
X = df_clean.drop(['anomaly'])

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [ ]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

In [ ]:
clf = LogisticRegression(class_weight='balanced', random_state=42, n_jobs=-1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['Benign', 'Malicious']))

In [ ]:
cf_matrix = confusion_matrix(y_test, y_pred)
sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, fmt='.2%', cmap='Blues')
plt.show()

### Downsample Dataset

In [ ]:
df_malicious = df_clean.filter(pl.col('anomaly') == 1)
df_benign    = df_clean.filter(pl.col('anomaly') == 0)

df_benign_downsample = resample(df_benign, replace=True, 
n_samples=len(df_malicious), random_state=42)

df_downsampled = pl.concat([df_malicious, df_benign_downsample])

In [ ]:
df_downsampled_malicious = df_downsampled.filter(pl.col('anomaly') == 1)
df_downsampled_benign    = df_downsampled.filter(pl.col('anomaly') == 0)

df_count = pl.DataFrame({'anomaly': ['Benign', 'Malicious'], 'count': [df_downsampled_benign.shape[0], df_downsampled_malicious.shape[0]]})
sns.barplot(x='anomaly', y='count', hue='anomaly', data=df_count)
plt.show()

In [ ]:
Y = df_downsampled['anomaly']
X = df_downsampled.drop(['anomaly'])

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [ ]:
clf = LogisticRegression(class_weight='balanced', random_state=42, n_jobs=-1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['Benign', 'Malicious']))

In [ ]:
cf_matrix = confusion_matrix(y_test, y_pred)
sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, fmt='.2%', cmap='Blues')
plt.show()

## Packet Flows

In [ ]:
def dataframe_shift(df, columns, windows):
    for i in range(1, windows):
        df = df.with_columns((pl.col(columns).shift(i)).name.prefix(f'prev_{i}_'))
    return df.drop_nulls()

In [ ]:
window = 5
df_users = df_clean.partition_by(by='user')
for i in range(len(df_users)):
    df_aux = df_users[i]
    df_users[i] = dataframe_shift(df_aux, columns=['dst_port', 'country'], windows=window)
df_clean_roll = pl.concat(df_users)
df_clean_roll.head()

### Full Dataset

In [ ]:
df_roll_malicious = df_clean_roll.filter(pl.col('anomaly') == 1)
df_roll_benign    = df_clean_roll.filter(pl.col('anomaly') == 0)

df_count = pl.DataFrame({'anomaly': ['Benign', 'Malicious'], 'count': [df_roll_benign.shape[0], df_roll_malicious.shape[0]]})
sns.barplot(x='anomaly', y='count', hue='anomaly', data=df_count)
plt.show()

In [ ]:
Y = df_clean['anomaly']
X = df_clean.drop(['anomaly'])

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [ ]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

In [ ]:
clf = LogisticRegression(class_weight='balanced', random_state=42, n_jobs=-1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['Benign', 'Malicious']))

In [ ]:
cf_matrix = confusion_matrix(y_test, y_pred)
sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, fmt='.2%', cmap='Blues')
plt.show()

### Downsample Dataset

In [ ]:
df_malicious = df_clean_roll.filter(pl.col('anomaly') == 1)
df_benign    = df_clean_roll.filter(pl.col('anomaly') == 0)

df_benign_downsample = resample(df_benign, replace=True, 
n_samples=len(df_malicious), random_state=42)

df_downsampled = pl.concat([df_malicious, df_benign_downsample])

In [ ]:
df_roll_downsampled_malicious = df_downsampled.filter(pl.col('anomaly') == 1)
df_roll_downsampled_benign    = df_downsampled.filter(pl.col('anomaly') == 0)

df_count = pl.DataFrame({'anomaly': ['Benign', 'Malicious'], 'count': [df_roll_downsampled_benign.shape[0], df_roll_downsampled_malicious.shape[0]]})
sns.barplot(x='anomaly', y='count', hue='anomaly', data=df_count)
plt.show()

In [ ]:
Y = df_downsampled['anomaly']
X = df_downsampled.drop(['anomaly'])

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [ ]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

In [ ]:
clf = LogisticRegression(class_weight='balanced', random_state=42, n_jobs=-1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['Benign', 'Malicious']))

In [ ]:
cf_matrix = confusion_matrix(y_test, y_pred)
sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, fmt='.2%', cmap='Blues')
plt.show()

## Outlier Detection Data Preprocessing

In [ ]:
df_clean_roll_malicious = df_clean_roll.filter(pl.col('anomaly') == 1)
df_clean_roll_benign    = df_clean_roll.filter(pl.col('anomaly') == 0)

Y = df_clean_roll_benign['anomaly']
X = df_clean_roll_benign.drop(['anomaly'])

k=3

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=df_clean_roll_malicious.shape[0]*k, random_state=42)

# Append the anomalies to the test
y_test = pl.concat([y_test, df_clean_roll_malicious['anomaly']])
X_test = pl.concat([X_test, df_clean_roll_malicious.drop(['anomaly'])])

In [ ]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

In [ ]:
X_train, X_validation = train_test_split(X_train, test_size=0.2, random_state=42)

## Traditional Outlier Detection

In [ ]:
clf = IsolationForest(n_estimators=100, n_jobs=-1, random_state=42)

clf.fit(X_train)
y_pred = clf.predict(X_test)
y_pred = np.clip(y_pred*-1, 0, 1)

print(classification_report(y_test, y_pred))

In [ ]:
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm/np.sum(cm), annot=True, fmt='.2%', cmap='Blues')
plt.show()

## Clustering

In [ ]:
sse = {}

for k in range(2, 22):
    kmeans = KMeans(n_clusters=k, n_init='auto', random_state=42)
    kmeans.fit(X_train)
    sse[k] = kmeans.inertia_

plt.title('Elbow plot for K selection')
plt.xlabel('k')
plt.ylabel('SSE')
sns.pointplot(x=list(sse.keys()), y=list(sse.values()))
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=10, n_init='auto', random_state=42)
kmeans.fit(X_train)

In [ ]:
distances = kmeans.transform(X_test)
cluster_indices = kmeans.predict(X_test)
mask = (cluster_indices[:, None] == np.arange(distances.shape[1]))
distances[~mask] = 0

dictionary = {i: column for i, column in enumerate(zip(*distances))}

mean_distances = [sum(values) / len(values) for key, values in dictionary.items()]
thresholds = np.array(mean_distances)

print(f'thresholds: {thresholds}')
k=3.0
outliers = np.any(np.array(distances) > k*thresholds, axis=1)
y_pred = [1 if v else 0 for v in outliers]
print(f'{outliers}')
print(f'{y_pred}')

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm/np.sum(cm), annot=True, fmt='.2%', cmap='Blues')
plt.show()

## AutoEncoder

In [ ]:
def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [ ]:
# data dimensions // hyperparameters 
input_dim = X_train.shape[1]
latent_dim = 6

encoder = tf.keras.models.Sequential([
    tf.keras.layers.Dense(12, kernel_initializer='glorot_uniform', activation='elu', input_shape=(input_dim,)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(latent_dim, kernel_initializer='glorot_uniform', activation='elu'),
])

decoder = tf.keras.models.Sequential([
    tf.keras.layers.Dense(12, kernel_initializer='glorot_uniform', activation='elu', input_shape=(latent_dim,)),
    tf.keras.layers.Dense(input_dim, kernel_initializer='glorot_uniform', activation='linear')
])

autoencoder = tf.keras.Model(inputs=encoder.input, outputs=decoder(encoder.output))
autoencoder.compile(optimizer='adam', loss='mse', metrics=[coeff_determination])
autoencoder.summary()

In [ ]:
EPOCHS = 30
BATCH_SIZE = 4096
history = autoencoder.fit(X_train, X_train, epochs=EPOCHS, batch_size=BATCH_SIZE, 
validation_data=(X_validation, X_validation), verbose=0, callbacks=[TqdmCallback(verbose=0)])

In [ ]:
# summarize history for accuracy
plt.plot(history.history['coeff_determination'])
plt.plot(history.history['val_coeff_determination'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower right')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [ ]:
reconstructions = autoencoder.predict(X_test)
mse = np.mean(np.power(X_test - reconstructions, 2), axis=1)

In [ ]:
malicious = mse[y_test==1]
benign    = mse[y_test==0]

fig, ax = plt.subplots(figsize=(6,6))

ax.hist(malicious, bins=30, density=True, label="malicious", alpha=.6, color="red")
ax.hist(benign, bins=30, density=True, label="benign", alpha=.6, color="green")

plt.title("(Normalized) Distribution of the Reconstruction Loss")
plt.legend()
plt.show()

In [ ]:
df_scores = pl.DataFrame({'mse': mse,'label': y_test})
ax = sns.violinplot(x='label', y='mse', cut = 0, data=df_scores)
ax.set_xticks([0,1])
ax.set_xticklabels(['Benign', 'Malicious'])
plt.show()

### Visualization of the class grouping

In [ ]:
from sklearn.manifold import TSNE

def tsne_scatter(features, labels):
    # t-SNE dimensionality reduction
    features_embedded = TSNE(n_components=2, random_state=42).fit_transform(features)
    
    # initialising the plot
    fig, ax = plt.subplots(figsize=(8,8))

    # plotting data
    ax.scatter(*zip(*features_embedded[np.where(labels==1)]), marker='o',color='r', s=2,alpha=0.7,label='Malicious')
    ax.scatter(*zip(*features_embedded[np.where(labels==0)]), marker='o',color='g',s=2, alpha=0.3,label='Benign')

    plt.show()

In [ ]:
tsne_scatter(X_test, y_test)

In [ ]:
X_Latent = encoder.predict(X_test)
tsne_scatter(X_Latent, y_test)

### Outlier identification and classification

In [ ]:
def mad_score(points):
    m = np.median(points)
    ad = np.abs(points - m)
    mad = np.median(ad)
    return 0.6745 * ad / mad

THRESHOLD = 3.5
z_scores = mad_score(mse)
outliers = z_scores > THRESHOLD

In [ ]:
print(classification_report(y_test, outliers, target_names=['Benign', 'Malicious']))

In [ ]:
cm = confusion_matrix(y_test, outliers)
sns.heatmap(cm/np.sum(cm), annot=True, fmt='.2%', cmap='Blues')
plt.show()